In [1]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# Plimpton 322 の解読 (2)

## YBC 6967 に基づく解釈

$c$を定数とする二次方程式

$$
x - \frac{1}{x} = c
$$

について、
$v_{1} = \frac{c}{2}$、
$v_{3} = 1 + {v_{1}}^2 = 1 + (\frac{c}{2})^2$、
$v_{4} = \sqrt{v_{3}} = \sqrt{1 + (\frac{c}{2})^2}$ と置くと、次の連立方程式が成り立つ:

$$
  \begin{align}
    x &= v_{4} + v_{1} \\
    \frac{1}{x} &= v_{4} - v_{1}
  \end{align}
$$

さらに、この連立方程式を$v_{1}$, $v_{4}$ について解くと、

$$
  \begin{align}
    v_{1} &= \frac{1}{2}(x-\frac{1}{x}) \\
    v_{4} &= \frac{1}{2}(x+\frac{1}{x})
  \end{align}
$$

である。

$v_{1}$, $1$, $v_{4}$ はピタゴラスの三数であり、${v_{1}}^2 + 1^2 = {v_{4}}^2$が成り立つ。

* [Pythagorean triple - Wikipedia](https://en.wikipedia.org/wiki/Pythagorean_triple)

後で説明する $a$ という係数を用いると、Plimpton 322は次表のように解釈できる:

|#1 |#2 |#3 |
|:-:|:-:|:-:|
|$$1+{v_{1}}^2 = {v_{4}}^2$$|$$a\cdot v_{1}$$|$$a\cdot v_{4}$$|
|$$1+\left(\frac{c}{2}\right)^2$$|$$\frac{a}{2}\left(x-\frac{1}{x}\right)$$|$$\frac{a}{2}\left(x+\frac{1}{x}\right)$$|

10進法で表記したPlimpton 322の11行目に着目する:

In [ ]:
import pandas as pd
import requests

res = requests.get("https://en.wikipedia.org/wiki/Plimpton_322").text

df2=pd.read_html(res)[1]
df2.take([10])

係数 $a$ は 60進法の $1\ 0$:

In [ ]:
import math
math.sqrt(v4**2 - v1**2)

In [ ]:
v1, v4 = 45, 75

In [ ]:
c = v1*2
c

In [ ]:
(v1/60), (v4/60)

In [ ]:
(v1/60)**2, (v4/60)**2

$x$ と $\frac{1}{x}$ を求める:

In [ ]:
v4+v1, v4-v1

In [ ]:
def inverse(n, base=10):
    denominator = n
    remainder   = 1 # initial remainder
    quotient    = 0 # dummy
    repeat = set()
    l = []
    while not quotient in repeat:
        repeat.add(quotient)
        numerator = remainder * base
        quotient, remainder = divmod(numerator, denominator)
        l.append(quotient)
        if remainder == 0:
            l.append(0)
            break
    return l

In [ ]:
inverse(2, 60), inverse(30, 60)